# Here i use sqlite instead of DB2

In [ ]:
#Install & load sqlite3
#!pip install sqlite3  ##Uncomment the code to install sqlite3
import sqlite3

In [3]:
import pandas as pd
space=pd.read_csv('Spacex.csv')

In [4]:
space.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Date               101 non-null    object
 1   Time (UTC)         101 non-null    object
 2   Booster_Version    101 non-null    object
 3   Launch_Site        101 non-null    object
 4   Payload            101 non-null    object
 5   PAYLOAD_MASS__KG_  101 non-null    int64 
 6   Orbit              101 non-null    object
 7   Customer           101 non-null    object
 8   Mission_Outcome    101 non-null    object
 9   Landing _Outcome   101 non-null    object
dtypes: int64(1), object(9)
memory usage: 8.0+ KB


In [9]:
#create an sqllite database
from pathlib import Path
Path('spacex.db').touch()

In [10]:
#Create a database connection and cursor to execute queries

conn = sqlite3.connect('spacex.db')
c = conn.cursor()

In [11]:
# Drop the table if already exists.
c.execute("DROP TABLE IF EXISTS space")

In [12]:

# write the data to a sqlite table
space.to_sql('space', conn, if_exists='append', index = False)

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [54]:
#List the attributs' type.
c.execute('''pragma table_info('space')''')
c.fetchall()

[(0, 'Date', 'TEXT', 0, None, 0),
 (1, 'Time (UTC)', 'TEXT', 0, None, 0),
 (2, 'Booster_Version', 'TEXT', 0, None, 0),
 (3, 'Launch_Site', 'TEXT', 0, None, 0),
 (4, 'Payload', 'TEXT', 0, None, 0),
 (5, 'PAYLOAD_MASS__KG_', 'INTEGER', 0, None, 0),
 (6, 'Orbit', 'TEXT', 0, None, 0),
 (7, 'Customer', 'TEXT', 0, None, 0),
 (8, 'Mission_Outcome', 'TEXT', 0, None, 0),
 (9, 'Landing _Outcome', 'TEXT', 0, None, 0)]

### Note
#### In order to modify the type of the date in date I copy the space table into a space3. space3 is a new table created originally empty with the desired columns type

In [67]:
c.executescript(""" 
    CREATE TABLE space3 (
            Date Date ,Time VARCHAR(255),'Booster_Version' VARCHAR(255),'Launch_Site' VARCHAR(255),
            'Payload' VARCHAR(255), 'PAYLOAD_MASS__KG_' VARCHAR(255), 'Orbit' VARCHAR(255), 'Customer' VARCHAR(255), 
             'Mission_Outcome' VARCHAR(255), 'Landing _Outcome' VARCHAR(255));
    INSERT INTO space3 SELECT * FROM space;
""")
  
# fetch the table data
print("Table data :")
c.execute('''DROP TABLE space''')
print(c.fetchall())

Table data :
[]


In [69]:
c.execute('''pragma table_info('space3')''')
c.fetchall()

[(0, 'Date', 'Date', 0, None, 0),
 (1, 'Time', 'VARCHAR(255)', 0, None, 0),
 (2, 'Booster_Version', 'VARCHAR(255)', 0, None, 0),
 (3, 'Launch_Site', 'VARCHAR(255)', 0, None, 0),
 (4, 'Payload', 'VARCHAR(255)', 0, None, 0),
 (5, 'PAYLOAD_MASS__KG_', 'VARCHAR(255)', 0, None, 0),
 (6, 'Orbit', 'VARCHAR(255)', 0, None, 0),
 (7, 'Customer', 'VARCHAR(255)', 0, None, 0),
 (8, 'Mission_Outcome', 'VARCHAR(255)', 0, None, 0),
 (9, 'Landing _Outcome', 'VARCHAR(255)', 0, None, 0)]

In [68]:
statement = '''SELECT * FROM space3'''
c.execute(statement)

print("All the data")
output_all = c.fetchall()
for row_all in output_all:
  print(row_all)

All the data
('04-06-2010', '18:45:00', 'F9 v1.0  B0003', 'CCAFS LC-40', 'Dragon Spacecraft Qualification Unit', '0', 'LEO', 'SpaceX', 'Success', 'Failure (parachute)')
('08-12-2010', '15:43:00', 'F9 v1.0  B0004', 'CCAFS LC-40', 'Dragon demo flight C1, two CubeSats, barrel of Brouere cheese', '0', 'LEO (ISS)', 'NASA (COTS) NRO', 'Success', 'Failure (parachute)')
('22-05-2012', '07:44:00', 'F9 v1.0  B0005', 'CCAFS LC-40', 'Dragon demo flight C2', '525', 'LEO (ISS)', 'NASA (COTS)', 'Success', 'No attempt')
('08-10-2012', '00:35:00', 'F9 v1.0  B0006', 'CCAFS LC-40', 'SpaceX CRS-1', '500', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')
('01-03-2013', '15:10:00', 'F9 v1.0  B0007', 'CCAFS LC-40', 'SpaceX CRS-2', '677', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')
('29-09-2013', '16:00:00', 'F9 v1.1  B1003', 'VAFB SLC-4E', 'CASSIOPE', '500', 'Polar LEO', 'MDA', 'Success', 'Uncontrolled (ocean)')
('03-12-2013', '22:41:00', 'F9 v1.1', 'CCAFS LC-40', 'SES-8', '3170', 'GTO', 'SES', 'S

In [19]:
#pd.read_sql('''SELECT * FROM space ''', conn)

### Task 1
#### Display the names of the unique launch sites in the space mission

In [70]:
#Display the names of the unique launch sites in the space mission
c.execute('''SELECT distinct(Launch_Site) FROM space3''')
c.fetchall()

[('CCAFS LC-40',), ('VAFB SLC-4E',), ('KSC LC-39A',), ('CCAFS SLC-40',)]

### Task 2
#### Display 5 records where launch sites begin with the string 'CCA'

In [71]:
#Display 5 records where launch sites begin with the string 'CCA'
condi='%CCA,'
q='SELECT * FROM space3 where Launch_Site like ?'
c.execute('''SELECT * FROM space3  where Launch_Site LIKE 'CCAFS%' Limit 5''')
#c.execute(q,condi)
#c.fetchall()
output_all = c.fetchall()
for row_all in output_all:
  print(row_all)

('04-06-2010', '18:45:00', 'F9 v1.0  B0003', 'CCAFS LC-40', 'Dragon Spacecraft Qualification Unit', '0', 'LEO', 'SpaceX', 'Success', 'Failure (parachute)')
('08-12-2010', '15:43:00', 'F9 v1.0  B0004', 'CCAFS LC-40', 'Dragon demo flight C1, two CubeSats, barrel of Brouere cheese', '0', 'LEO (ISS)', 'NASA (COTS) NRO', 'Success', 'Failure (parachute)')
('22-05-2012', '07:44:00', 'F9 v1.0  B0005', 'CCAFS LC-40', 'Dragon demo flight C2', '525', 'LEO (ISS)', 'NASA (COTS)', 'Success', 'No attempt')
('08-10-2012', '00:35:00', 'F9 v1.0  B0006', 'CCAFS LC-40', 'SpaceX CRS-1', '500', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')
('01-03-2013', '15:10:00', 'F9 v1.0  B0007', 'CCAFS LC-40', 'SpaceX CRS-2', '677', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')


### Task 3
#### Display the total payload mass carried by boosters launched by NASA (CRS)

In [72]:
#Display the total payload mass carried by boosters launched by NASA (CRS)
c.execute('''SELECT sum(PAYLOAD_MASS__KG_) as total_payload FROM space3  where Customer='NASA (CRS)' ''')
c.fetchall()

[(45596,)]

### Task 4
#### Display average payload mass carried by booster version F9 v1.1

In [73]:
#Display average payload mass carried by booster version F9 v1.1
c.execute('''SELECT avg(PAYLOAD_MASS__KG_) as avg_payload FROM space3  where  Booster_Version='F9 v1.1' ''')
c.fetchall()

[(2928.4,)]

### Task 5
#### List the date when the first successful landing outcome in ground pad was acheived.
##### Hint:Use min function

In [81]:
#List the date when the first successful landing outcome in ground pad was acheived.
c.execute('''select min (Date) from space3 where [Landing _Outcome] Like 'Success %'
''')
c.fetchall()

[('01-05-2017',)]

### Task 6
#### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [75]:

c.execute('''select  * from space3
where [Landing _Outcome]='Success (drone ship)'
and
PAYLOAD_MASS__KG_ between 4000 and 6000''')
output_all = c.fetchall()
for row_all in output_all:
  print(row_all)


('06-05-2016', '05:21:00', 'F9 FT B1022', 'CCAFS LC-40', 'JCSAT-14', '4696', 'GTO', 'SKY Perfect JSAT Group', 'Success', 'Success (drone ship)')
('14-08-2016', '05:26:00', 'F9 FT B1026', 'CCAFS LC-40', 'JCSAT-16', '4600', 'GTO', 'SKY Perfect JSAT Group', 'Success', 'Success (drone ship)')
('30-03-2017', '22:27:00', 'F9 FT  B1021.2', 'KSC LC-39A', 'SES-10', '5300', 'GTO', 'SES', 'Success', 'Success (drone ship)')
('24-08-2017', '18:51:00', 'F9 FT B1038.1', 'VAFB SLC-4E', 'Formosat-5', '475', 'SSO', 'NSPO', 'Success', 'Success (drone ship)')
('11-10-2017', '22:53:00', 'F9 FT  B1031.2', 'KSC LC-39A', 'SES-11 / EchoStar 105', '5200', 'GTO', 'SES EchoStar', 'Success', 'Success (drone ship)')


### Task 7
#### List the total number of successful and failure mission outcomes

In [76]:
c.execute('''select  count(Mission_Outcome) as Mission_outcome_total from space3
''')
output_all = c.fetchall()
for row_all in output_all:
  print(row_all)


(101,)


### Task 8
#### List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [77]:
c.execute('''select Booster_Version from space3
where PAYLOAD_MASS__KG_=(select max(PAYLOAD_MASS__KG_) from space)
''')
output_all = c.fetchall()
for row_all in output_all:
  print(row_all)

('F9 B5 B1048.4',)
('F9 B5 B1049.4',)
('F9 B5 B1051.3',)
('F9 B5 B1056.4',)
('F9 B5 B1048.5',)
('F9 B5 B1051.4',)
('F9 B5 B1049.5',)
('F9 B5 B1060.2 ',)
('F9 B5 B1058.3 ',)
('F9 B5 B1051.6',)
('F9 B5 B1060.3',)
('F9 B5 B1049.7 ',)


### Task 9
#### List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015

In [103]:
c.execute('''select Date, Booster_Version, Launch_Site from space3 as s
where [Landing _Outcome]='Failure (drone ship)'
and strftime('%Y', s.Date)='2015'
''')

output_all = c.fetchall()
for row_all in output_all:
  print(row_all)

### Task 10
#### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order

In [94]:
c.execute('''select  * from space3
where [Landing _Outcome] in ('Failure (drone ship)','Success (ground pad)')
and Date between '04-06-2010' and '20-03-2017'
order by Date desc
''')
output_all = c.fetchall()
for row_all in output_all:
  print(row_all)

('19-02-2017', '14:39:00', 'F9 FT B1031.1', 'KSC LC-39A', 'SpaceX CRS-10', '2490', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (ground pad)')
('18-07-2016', '04:45:00', 'F9 FT B1025.1', 'CCAFS LC-40', 'SpaceX CRS-9', '2257', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (ground pad)')
('17-01-2016', '18:42:00', 'F9 v1.1 B1017', 'VAFB SLC-4E', 'Jason-3', '553', 'LEO', 'NASA (LSP) NOAA CNES', 'Success', 'Failure (drone ship)')
('15-12-2017', '15:36:00', 'F9 FT  B1035.2', 'CCAFS SLC-40', 'SpaceX CRS-13', '2205', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (ground pad)')
('15-06-2016', '14:29:00', 'F9 FT B1024', 'CCAFS LC-40', 'ABS-2A Eutelsat 117 West B', '3600', 'GTO', 'ABS Eutelsat', 'Success', 'Failure (drone ship)')
('14-08-2017', '16:31:00', 'F9 B4 B1039.1', 'KSC LC-39A', 'SpaceX CRS-12', '3310', 'LEO (ISS)', 'NASA (CRS)', 'Success', 'Success (ground pad)')
('14-04-2015', '20:10:00', 'F9 v1.1 B1015', 'CCAFS LC-40', 'SpaceX CRS-6', '1898', 'LEO (ISS)', 'NASA (CRS)', 'Success', 